In [47]:
import coolsearch.utility_functions as util
import coolsearch.models as cmodel
import coolsearch.search as csearch
from importlib import reload
from time import sleep

import numpy as np
import polars as pl
from plotly import graph_objects as go

util.set_plotly_template()

reload(cmodel)
reload(csearch)
reload(util)

<module 'coolsearch.utility_functions' from 'C:\\Users\\marcu\\Documents\\RANDO_PROJECTS\\cool_search\\coolsearch\\utility_functions.py'>

In [48]:
f = util.test_function_01

xx = np.linspace(-10, 10, 10000)
yy = f(xx)

idx_min_true = np.argmin(yy)
print("true min",xx[idx_min_true], yy[idx_min_true])

true min 1.6371637163716368 1.7939680806051224


In [49]:
reload(cmodel)

samples = pl.DataFrame({"x": np.linspace(-10, 10, 5)})
samples = samples.with_columns(
    pl.col("x").map_elements(f, return_dtype=pl.Float64).alias("f"),
)


model = cmodel.PolynomialModel(
    samples=samples,
    features=["x"],
    degree=2,
    target="f",
)
model.fit()

print(model.polynomial)
f_pred=model.predict()

f_pred_poly = model.polynomial(samples["x"])
print(f_pred_poly)

samples = samples.with_columns(f_pred=f_pred)
display(samples)
min_poly, poly_val = model.minimum_numeric(1000)
display(min_poly)

5 samples
3 poly features
coefficients: [ 2.87147841 -0.29012433  0.07286349], residuals: [0.8074821]
2.87147841 - 0.29012433 x + 0.07286349 x**2
[13.05907123  6.14368744  2.87147841  3.24244412  7.25658458]


x,f,f_pred
f64,f64,f64
-10.0,13.000519,13.059071
-5.0,6.472596,6.143687
0.0,2.236068,2.871478
5.0,3.76075,3.242444
10.0,7.103333,7.256585


1 -> 3 features:


x,value
f64,f64
1.991992,2.582678


In [52]:
fig = go.Figure(layout=dict(title="Polynomial model"))
fig.add_traces([
    go.Scatter(x=samples["x"],y=samples["f"],mode="markers",name = "input"),
    go.Scatter(x=poly_val["x"],y=poly_val["value"],mode="lines",name="fitted poly"),
    go.Scatter(x=xx,y=yy,mode="lines",name="true"),
    ]
)

In [51]:
model.residuals

array([0.8074821])